In [1]:
import pandas as pd

Utilities

In [89]:
def get_filename(key):
    return f"RB_DishwashWeekly_GB_{key}_07072021.csv"

def explore(name, df: pd.DataFrame):
    print(f"Exploration of {name}")
    print(f"Shape: {df.shape}")
    df.info()
    print(df.describe())
    print("------------------------------------------------------")

def clean(df: pd.DataFrame):
    df.dropna()
    df.drop_duplicates()

load csv files into dataFrame to explore

In [40]:
dim_df = pd.read_csv(get_filename("DIM"), delimiter="|")

fct_df = pd.read_csv(get_filename("FCT"), delimiter="|")
mkt_df = pd.read_csv(get_filename("MKT"), delimiter="|")
per_df = pd.read_csv(get_filename("PER"), delimiter="|")
prod_df = pd.read_csv(get_filename("PROD"), delimiter="|")

Explore dataFrames

In [ ]:
explore("dimension", dim_df)
explore("fact", fct_df)
explore("market", mkt_df)
explore("period", per_df)
explore("product", prod_df)

Clean dataFrames

In [57]:
clean(mkt_df)
clean(per_df)
clean(prod_df)

Our fact_data is too large, so we use chunk to transform it, and retrieve insights

In [1]:
rename_criteria = {row["TAG"]: row["SHORT"] for _, row in fct_df.iterrows()}
print(rename_criteria)

def process(chunk: pd.DataFrame):
    clean(chunk)
    # rename fact columns
    chunk.rename(columns=rename_criteria, inplace=True)
    chunk = chunk.merge(mkt_df, left_on="MKT_TAG", right_on="TAG", how="left")\
                 .merge(per_df, left_on="PER_TAG", right_on="TAG", how="left")\
                 .merge(prod_df, left_on="PROD_TAG", right_on="TAG")
    explore("chunk", chunk)
    # we can add chunked data to other dataFrame or file or database here


NameError: name 'fct_df' is not defined

In [90]:
with pd.read_csv(get_filename("fact_data"), delimiter="|", chunksize=1000) as reader:
    for chunk in reader:
        process(chunk)

Exploration of chunk
Shape: (1000, 180)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 180 entries, MKT_TAG to RB SUMMARY SIZE RANGE
dtypes: float64(128), int64(8), object(44)
memory usage: 1.4+ MB
              Units        Volume         Value  ACV Distribution (w)  \
count  8.440000e+02  8.440000e+02  8.440000e+02            828.000000   
mean   7.467226e+04  2.944665e+06  3.978938e+05              9.280193   
std    4.371115e+05  2.265833e+07  2.212409e+06             15.673011   
min   -1.000000e+01 -1.000000e+03 -1.000000e+02              0.000000   
25%    1.792500e+02  4.320000e+03  1.009000e+03              0.000000   
50%    3.860000e+03  7.238500e+04  1.533850e+04              1.000000   
75%    4.829000e+04  1.188159e+06  2.608462e+05             12.000000   
max    1.011714e+07  5.760262e+08  4.457382e+07             86.000000   

       Numeric Distribution (w)  Weight. Distribution (w)  Base Units  \
count                828.000000     